In [1]:
import h5py
import numpy as np

 
def load_s(mat_file_path: str) -> np.ndarray:
    def _find_s(group: h5py.Group):
        if 's' in group:
            item = group['s']
            if isinstance(item, h5py.Dataset):
                return np.array(item)
            if isinstance(item, h5py.Group):
                for _, sub in item.items():
                    if isinstance(sub, h5py.Dataset):
                        return np.array(sub)
        for _, item in group.items():
            if isinstance(item, h5py.Group):
                out = _find_s(item)
                if out is not None:
                    return out
        return None 
    with h5py.File(mat_file_path, 'r') as f:
        s = _find_s(f)
        if s is None:
            raise KeyError("Variable 's' not found in the .mat file.")
        return s
 
mat_file_path = r"/storage/scratch1/1/bchoi73/filcoh_1e-5.mat"
 
Lx = 2*np.pi; Ly = 2*np.pi;
s = load_s(mat_file_path)
V = s[0, 0, :, :]  
U = s[0, 1, :, :]  
Nx , Ny = U.shape
dx = Lx/Nx; dy = Ly/Ny
 
print(f"V: Shape = {V.shape}, Type = {type(V)}")
print(f"U: Shape = {U.shape}, Type = {type(U)}")

V: Shape = (2048, 2048), Type = <class 'numpy.ndarray'>
U: Shape = (2048, 2048), Type = <class 'numpy.ndarray'>


In [2]:
def small_vortex_scales(U, V, Lx=2*np.pi, Ly=2*np.pi, remove_mean=True, remove_strips=False):
    if remove_mean:
        U = U - U.mean(); V = V - V.mean()
    Ny, Nx = U.shape

    kx = 2*np.pi*np.fft.fftfreq(Nx, d=Lx/Nx)
    ky = 2*np.pi*np.fft.fftfreq(Ny, d=Ly/Ny)
    KX, KY = np.meshgrid(kx, ky, indexing='xy')
    Uhat, Vhat = np.fft.fft2(U), np.fft.fft2(V)
    if remove_strips:
        Uhat[0,:] = 0; Uhat[:,0] = 0
        Vhat[0,:] = 0; Vhat[:,0] = 0
    what = 1j*KX*Vhat - 1j*KY*Uhat
    w = np.fft.ifft2(what).real

    Kmag   = np.sqrt(KX**2 + KY**2)
    Ek_hat = 0.5*(np.abs(Uhat)**2 + np.abs(Vhat)**2)
    k_phys = Kmag * (Lx/(2*np.pi))       
    kmax   = int(np.floor(k_phys.max()))
    edges  = np.arange(0.5, kmax+1.5, 1.0)
    idx    = np.digitize(k_phys, edges)
    nb     = len(edges)
    E_k    = np.bincount(idx.ravel(), weights=Ek_hat.ravel(), minlength=nb+1)[1:nb+1]
    k      = np.arange(1, nb+1)         

    sumE = E_k.sum()
    if sumE > 0:
        n_energy      = np.sum(k * E_k) / sumE
        n_energy_rms  = np.sqrt(np.sum((k**2) * E_k) / sumE)
    else:
        n_energy = np.nan
        n_energy_rms = np.nan

    Z_k = (k**2) * E_k
    n_omega_peak = k[np.argmax(Z_k)]
    sumZ = Z_k.sum()
    if sumZ > 0:
        n_omega     = np.sum(k * Z_k) / sumZ
        n_omega_rms = np.sqrt(np.sum((k**2) * Z_k) / sumZ)
    else:
        n_omega = np.nan
        n_omega_rms = np.nan

    def L_over(n): 
        return (Lx / n) if (np.isfinite(n) and (n > 0)) else np.nan

    l_energy      = L_over(n_energy)
    l_energy_rms  = L_over(n_energy_rms)
    l_omega       = L_over(n_omega)
    l_omega_rms   = L_over(n_omega_rms)
    lomega_peak   = L_over(n_omega_peak)

    return {
        "k_energy_centroid_n": float(n_energy),
        "k_energy_rms_n": float(n_energy_rms),
        "k_omega_centroid_n": float(n_omega),
        "k_omega_rms_n": float(n_omega_rms),
        "komega_peak_n": int(n_omega_peak),
        "l_energy_centroid": float(l_energy),
        "l_energy_rms": float(l_energy_rms),
        "l_omega_centroid": float(l_omega),
        "l_omega_rms": float(l_omega_rms),
        "lomega_peak": float(lomega_peak),
    }

out = small_vortex_scales(U, V, Lx=2*np.pi, Ly=2*np.pi)

l_omega       = out["l_omega_centroid"]
l_omega_rms   = out["l_omega_rms"]
l_energy      = out["l_energy_centroid"]
l_energy_rms  = out["l_energy_rms"]
n_omega       = out["k_omega_centroid_n"]
n_omega_rms   = out["k_omega_rms_n"]
n_energy      = out["k_energy_centroid_n"]
n_energy_rms  = out["k_energy_rms_n"]
n_omega_peak  = out["komega_peak_n"]

print(f"ENSTROPHY (indices):   n_ω ≈ {n_omega:.0f}, n_ω,rms ≈ {n_omega_rms:.0f}")
print(f"ENSTROPHY (lengths):   l_ω ≈ {l_omega:.3f}, l_ω,rms ≈ {l_omega_rms:.3f}")
print(f"ENERGY (indices):      n_E ≈ {n_energy:.0f}, n_E,rms ≈ {n_energy_rms:.0f}")
print(f"ENERGY (lengths):      l_E ≈ {l_energy:.3f}, l_E,rms ≈ {l_energy_rms:.3f}")

ENSTROPHY (indices):   n_ω ≈ 17, n_ω,rms ≈ 25
ENSTROPHY (lengths):   l_ω ≈ 0.380, l_ω,rms ≈ 0.250
ENERGY (indices):      n_E ≈ 1, n_E,rms ≈ 2
ENERGY (lengths):      l_E ≈ 4.882, l_E,rms ≈ 2.979
